# Import packages

In [1]:
import pandas as pd
from teradataml import *
import ast
import json


from utils import clean_text
from constants import (

SHOPMAINA_DATASET, SKROUTZ_DATASET, PRICE_RUNNER_DATASET, JIO_MART_DATASET
)


In [14]:
jio_mart_df = pd.read_csv(JIO_MART_DATASET)
jio_mart_df = jio_mart_df.loc[~jio_mart_df['items'].isna(), :]
jio_mart_df = jio_mart_df.drop(columns=['href', 'price'])
jio_mart_df.rename(columns={'items': 'product name'}, inplace=True)
jio_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162280 entries, 0 to 162312
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   category      162280 non-null  object
 1   sub_category  162280 non-null  object
 2   product name  162280 non-null  object
dtypes: object(3)
memory usage: 5.0+ MB


In [12]:
jio_mart_df_cateogires = jio_mart_df["category"].unique()
jio_mart_df_cateogires

array(['Groceries', 'Home & Kitchen', 'Fashion', 'Electronics', 'Beauty',
       'Jewellery'], dtype=object)

In [13]:
jio_mart_df_cateogires = jio_mart_df["sub_category"].unique()
jio_mart_df_cateogires

array(['Fruits & Vegetables', 'Premium Fruits', 'Dairy & Bakery',
       'Staples', 'Snacks & Branded Foods', 'Beverages', 'Personal Care',
       'Home Care', 'Apparel', 'Mom & Baby Care', 'Books', 'Pets',
       'Kitchenware', 'Dining', 'Furnishing', 'Home Decor', 'Furniture',
       'Home Appliances', 'Toys, Games & Fitness', 'Electrical',
       'Bathroom & Laundry Accessories', 'Disposables', 'Stationery',
       'Bags & Travel Luggage', 'Mops, Brushes & Scrubs', 'Auto Care',
       'Garden & Outdoor', 'Carpentry & work accessories', 'Pooja Needs',
       'Bathroom & Laundry', 'Industrial & Scientific Supplies',
       'Building Supplies & Measuring Tools', 'Hardware & Plumbing',
       'Home Safety & Automation', 'Kitchen & Bath Fixtures',
       'Paint, Wall Treatments & Supplies', 'Power & Hand Tools',
       'Handloom & Handicraft', 'Personal Wear', 'Men', 'Women', 'Boys',
       'Girls', 'Junior Boys', 'Infants', 'Mobiles & Tablets',
       'TV & Speaker', 'Computers', 'Camer